In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from preprocess_functions import preprocess #preprocess is a class containing all the function

## Set up paths and params, then load data and fill object
path is the root_data path as defined in https://github.com/ikharitonov/vestibular_vr_pipeline/issues/25)

Select sensors if sensor-specific (and not "auto") filtering is used. 'G8m', 'g5-HT3', 'rG1' or available sensors in the function, otherwise asks for user input for half decay time in ms.

Target area is the intended area, not verified by histology yet. Added to self.info dictionary.  

In [ ]:
path = '/Users/rancze/Documents/Data/vestVR/Cohort2_test/2025-02-13T12-41-57'
sensors = {'470':'g5-HT3', '560':'rG1', '410':'isosbestic'}
plot_info = (' ') # can add addition info here for figure title, e.g. retro inj in SC
target_area = ('X') #ASSUMES the same target area for all folders, DANGER
filtering_method = 'auto' # auto (nyquist, recommended) or sensor 
detrend_method = 'divisive' #subtractive or divisive (recommended)
motion_correction = False # we never really use it
iso_channel = 410 # channel to be used for isosbestic control
signal_channel = 470 # channel to motion correct with isosbestic 

In [ ]:
#Create an object which will contain an increasing amount of information as functions are called on
processed = preprocess(path, sensors)
# extract all relevant and irrelevant info from the Fluorescence.csv file which contains the metadata 
processed.info = processed.get_info()

In [ ]:
#Loads Events.csv and Fluorescence-unaligned.csv
#Aligns to 470 nm timestamps (assumes 470 exists) and cuts data if needed (almost never)
#Returns processed dataframes below
(
    processed.rawdata, 
    processed.data, 
    processed.data_seconds, 
    processed.signals, 
) = processed.create_basic(
    cutstart = False,
    cutend = False,
    target_area = target_area, 
    motion = motion_correction
)
print ("Done")

### Filtering
All the sigals are low pass filtered using a butterworth filter.  
method = "auto" cutoff frequncy ~sample_rate/2 Hz  
method = "sensor" cutoff frequency is determined in the function using the sensors dictionary  

In [ ]:
processed.filtered = processed.low_pass_filt(method = filtering_method, plot=True)

### Detrending
A double exponential fit is made to account for sources of bleaching and the signal is corrected.  
method = "subtractive" assumes bleaching is sensor-independent (e.g. autofluorescence)  
method = "divisive" assumes bleaching comes from the sensor. This is most plausible.   
**N.B.** divisive detrended data is already dF/F. 

In [ ]:
processed.detrended, processed.exp_fits = processed.detrend(plot = True, method = detrend_method)

### Motion correction
iso_channel = channel to be used for isosbestic control

signal_channel = channel to motion correct with isosbestic 

In [ ]:
processed.motion_corrected = processed.motion_correct(plot = True, iso_ch = iso_channel, signal_ch = signal_channel)

### Delta F / F
This is a standard way of calculating the detla F over F signal based on the fiber photometry primer paper code: https://github.com/ThomasAkam/photometry_preprocessing/blob/master/Photometry%20data%20preprocessing.ipynb
With divisive detrending, this has already been calculated and the code treats it accordingly. 

In [ ]:
processed.deltaF_F = processed.get_deltaF_F(plot = True)

### Z-scoring
Standard Z-scoring of the dF/F

In [ ]:
processed.zscored = processed.z_score(plot = True)

In [ ]:
processed.cross_correlate_signals(col1='470', col2='560', plot=True)

### Save info, processed fluorescence and original events as a .csv files

In [ ]:
#again it ensures that the folder to save in already exists, since the csv must have somewhere to be
processed.info_csv = processed.write_info_csv()
processed.data_csv = processed.write_preprocessed_csv() #optional: Events = True; motion = False not impleneted yet
#optional:, motion_correct = True, Onix_align =False

In [ ]:
processed.plot_all_signals(sensors, plot_info)